In [1]:
#hide
%load_ext autoreload
%autoreload 2

In [2]:
import yaml

#export
class AppConfig():
    def __init__(self, file_name):
        with open(file_name) as f:
            self.config = yaml.safe_load(f)
        self.database_user = self.config["database_user"]
        self.database_password = self.config["database_password"]
        self.database_name = self.config["database_name"]
        self.database_host = self.config["database_host"]
        self.database_port = self.config["database_port"]
        self.table_index_values_name = self.config["table_index_values_name"]
        self.table_base_rates_name = self.config["table_base_rates_name"]
        self.data_file_folder = self.config["data_file_folder"]
        
class DataFileContainer():
    def __init__(self, file_name):
        with open(file_name) as f:
            self.yaml_file = yaml.safe_load(f)
        self.index_value_files = self.yaml_file["index_data"]
        self.base_rate_files = self.yaml_file["base_rate_data"]

In [3]:
#hide
from nbdev.showdoc import *
from IPython.display import Markdown,display
#import nb2ltx.AppConfig as AppConfig
import pyorient
import pandas as pd
import os.path

In [4]:
config = AppConfig("config.yml")

In [5]:
client = pyorient.OrientDB(config.database_host, config.database_port)
client.connect(config.database_user, config.database_password)

10

In [6]:
if client.db_exists(config.database_name):
   # Drop Database
   client.db_drop(config.database_name)
    
# Create Database
client.db_create(
  config.database_name,
  pyorient.DB_TYPE_DOCUMENT,
  pyorient.STORAGE_TYPE_PLOCAL 
)

database = client.db_open(config.database_name, config.database_user, config.database_password)

In [7]:
client.command("CREATE CLASS " + config.table_index_values_name + " IF NOT EXISTS")
client.command("CREATE CLASS " + config.table_base_rates_name + " IF NOT EXISTS")

[12]

In [18]:
data_file_container = DataFileContainer("files.yml")

def prepare_index_data(df, isin, region):
    df.columns = df.columns.str.replace(' ','_')
    df.dropna(inplace = True)
    df["ISIN"] = isin
    df["Region"] = region

current_index = 1
    
for file in data_file_container.index_value_files:
    df = pd.read_csv(os.path.join(config.data_file_folder, file["file_name"]))
    prepare_index_data(df, file["isin"], file["region"])
    print("Inserting '" + file["file_name"] + "' (" + str(current_index) + " of " + str(len(data_file_container.index_value_files)) + ") ...")
    current_index = current_index + 1
    for index, row in df.iterrows():
        insert_command = "INSERT INTO " + config.table_index_values_name + " CONTENT " + row.to_json()
        client.command(insert_command)
print("Done!")

Inserting 'Index CAC40 (FCHI) Entwicklung.csv' (1 of 7) ...
Inserting 'Index DAX Entwicklung.csv' (2 of 7) ...
Inserting 'Index Dow Jones (DJI) Entwicklung.csv' (3 of 7) ...
Inserting 'Index Euronext 100 (N100) Entwicklung.csv' (4 of 7) ...
Inserting 'Index NASDAQ100 (NDX) Entwicklung.csv' (5 of 7) ...
Inserting 'Index S&P500 Entwicklung.csv' (6 of 7) ...
Inserting 'Index STOXX50E Entwicklung seit 2007.csv' (7 of 7) ...
Done!


In [36]:
print(client.query("SELECT FROM " + config.table_index_values_name, 10)[0])

{'@index_value':{'Date': '1987-12-30', 'Open': 1005.190002, 'High': 1005.190002, 'Low': 1005.190002, 'Close': 1005.190002, 'Adj_Close': 1005.190002, 'Volume': 0.0, 'ISIN': 'DE0008469008'},'version':1,'rid':'#25:0'}


In [11]:
# FED base rate
df = pd.read_excel('C:/Users/menze/OneDrive/Polipol-Laptop/Studium/21Q2/WDK/Projekt/Statista FED Leitzins Entwicklung.xlsx', 1)
df.columns = df.columns.str.replace(' ','_')
df.dropna(inplace = True)
df["Bank"] = "FED"

print(df)

for index, row in df.iterrows():
    insert_command = "INSERT INTO base_rate CONTENT " + row.to_json()
    print(insert_command)
    client.command(insert_command)

Empty DataFrame
Columns: [Unnamed:_0, Unnamed:_1, Unnamed:_2, Unnamed:_3, Bank]
Index: []
